In [7]:
import pandas as pd
file_path = "/Users/liuyuxuan/usc-ds560-25sp/lab3/raw_data/stock_history.csv"
df = pd.read_csv(file_path)
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17940 entries, 0 to 17939
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  17940 non-null  int64  
 1   stock_id            17940 non-null  int64  
 2   collected_interval  17940 non-null  object 
 3   date                17940 non-null  object 
 4   open_price          17940 non-null  float64
 5   high_price          17940 non-null  float64
 6   low_price           17940 non-null  float64
 7   close_price         17940 non-null  float64
 8   volume              17940 non-null  int64  
 9   dividend_amount     17940 non-null  float64
 10  stock_split         17940 non-null  float64
 11  insertion_time      17940 non-null  object 
dtypes: float64(6), int64(3), object(3)
memory usage: 1.6+ MB


(None,
    id  stock_id collected_interval        date  open_price  high_price  \
 0   1         1                 1d  2024-01-31      186.13      186.19   
 1   2         1                 1d  2024-02-01      183.09      186.04   
 2   3         1                 1d  2024-02-02      178.98      186.42   
 3   4         1                 1d  2024-02-05      187.23      188.33   
 4   5         1                 1d  2024-02-06      185.95      188.39   
 
    low_price  close_price     volume  dividend_amount  stock_split  \
 0     183.45       183.50   55467800              0.0          0.0   
 1     182.92       185.95   64885400              0.0          0.0   
 2     178.38       184.94  102518000              0.0          0.0   
 3     184.93       186.77   69668800              0.0          0.0   
 4     185.86       188.38   43490800              0.0          0.0   
 
         insertion_time  
 0  2025-01-31 09:40:31  
 1  2025-01-31 09:40:31  
 2  2025-01-31 09:40:31  
 3  2025-

In [8]:
import pandas as pd
import numpy as np

file_path = "/Users/liuyuxuan/usc-ds560-25sp/lab3/raw_data/stock_history.csv"
df = pd.read_csv(file_path)

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['stock_id', 'date']).reset_index(drop=True)

# Handle missing values
print("Missing values before filling:\n", df.isnull().sum())
df.fillna(method='ffill', inplace=True)  # Forward fill
df.fillna(method='bfill', inplace=True)  # Backward fill
df.dropna(inplace=True)  # Remove NaN

# Calculate daily returns (percentage change in close price)
df['daily_return'] = df.groupby('stock_id')['close_price'].pct_change()

# Compute log returns to handle extreme price variations
df['log_return'] = df.groupby('stock_id')['close_price'].transform(lambda x: np.log(x / x.shift(1)))

# Compute 5-day and 20-day moving averages
df['moving_avg_5d'] = df.groupby('stock_id')['close_price'].transform(lambda x: x.rolling(window=5).mean())
df['moving_avg_20d'] = df.groupby('stock_id')['close_price'].transform(lambda x: x.rolling(window=20).mean())

# Compute rolling volatility (5-day standard deviation of daily returns)
df['volatility_5d'] = df.groupby('stock_id')['daily_return'].transform(lambda x: x.rolling(window=5).std())

# Compute cumulative return (total return over time for each stock)
df['cumulative_return'] = df.groupby('stock_id')['daily_return'].cumsum()

# Compute volume change percentage
if 'volume' in df.columns:
    df['volume_change'] = df.groupby('stock_id')['volume'].pct_change()

df = df[df['close_price'] > 0]

output_path = "/Users/liuyuxuan/usc-ds560-25sp/lab3/processed_data/processed_stock_history.csv"
df.to_csv(output_path, index=False)
print("Processed stock data saved to:", output_path)

Missing values before filling:
 id                    0
stock_id              0
collected_interval    0
date                  0
open_price            0
high_price            0
low_price             0
close_price           0
volume                0
dividend_amount       0
stock_split           0
insertion_time        0
dtype: int64
Processed stock data saved to: /Users/liuyuxuan/usc-ds560-25sp/lab3/processed_data/processed_stock_history.csv


/var/folders/2w/fwzncnms3ql_w80rrdmjzw0c0000gn/T/ipykernel_40193/1365743532.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill missing values
/var/folders/2w/fwzncnms3ql_w80rrdmjzw0c0000gn/T/ipykernel_40193/1365743532.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)  # Backward fill missing values
